## Topics Chunking

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_core.documents import Document
from langchain_community.document_transformers import Html2TextTransformer
import os 

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
def chunk_in_topics(text):
    topics = text.split('##')
    topics = [topic.strip() for topic in topics if topic.strip()]
    
    return topics

def get_documents(topics):
    return [Document(
            page_content=topic,
            metadata={"source": "hotmart"},
            id=index,
        ) for index,topic in enumerate(topics)]
def load_documents(urls):
    loader = AsyncHtmlLoader(urls)
    return loader.load()

def transform_documents(docs):
    html2text = Html2TextTransformer()
    return "\n".join(doc.page_content for doc in html2text.transform_documents(docs)) 

In [ ]:
combined_content = "\n".join(doc.page_content for doc in docs_transformed)

In [3]:
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
docs = get_documents(chunk_in_topics(transform_documents(load_documents(["https://hotmart.com/pt-br/blog/como-funciona-hotmart"]))))
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-3-small"))

Fetching pages: 100%|##########| 1/1 [00:02<00:00,  2.26s/it]


In [4]:
vectorstore.similarity_search("Um produtor pode ser um blogueiro?", k=2)

[Document(metadata={'source': 'hotmart'}, page_content='# **Quem pode se tornar um Produtor digital?**\n\nPode parecer clichê, mas a verdade é que **qualquer pessoa pode se tornar um\nProdutor digital** - basta que ela tenha como produzir seu material e saiba\nusar a internet.\n\nO diferencial para alcançar o êxito está na persistência do empreendedor e na\nvontade que ele tem de aprender com o próprio negócio.'),
 Document(metadata={'source': 'hotmart'}, page_content='**O que são Produtores?**\n\nAgora que você já entendeu o que é um produto digital, está na hora de\ndescobrir como você se encaixa nesse mercado.\n\nExistem, basicamente, dois tipos de pessoas que atuam nessa área, os\nProdutores e os Afiliados.\n\n**Os Produtores são pessoas que desejam criar algum tipo de conteúdo digital\npara ser vendido na internet.**\n\nO Produtor de conteúdo digital pode ser um autor independente, blogueiro ou\napenas alguém que domina um determinado assunto e decidiu criar um produto\npara atuar

In [5]:
vectorstore.similarity_search("Para quem a Hotmart é indicada?", k=2)

[Document(metadata={'source': 'hotmart'}, page_content='Para quem a Hotmart é indicada?\n\nComo você já viu até aqui, a Hotmart é a solução ideal para quem procura\nsucesso em negócios digitais.\n\nQualquer um pode investir no empreendedorismo digital e passar a viver de\ninternet. Alguns exemplos são:\n\n  * **Quem quer monetizar seu próprio conhecimento:** qualquer conteúdo pode ser transformado em negócio. Basta passear pelo marketplace da Hotmart. Você vai encontrar infoprodutos dos mais diferentes nichos, de agronegócios a idiomas, passando por artesanato e conserto de eletrodomésticos.\n  * **Trabalhar com marketing digital:** um modelo de negócio bastante popular é ter uma pessoa responsável pelo conteúdo dos produtos digitais e outra focada no lançamento. Aqui, estão envolvidas diferentes habilidades, como tráfego pago, SEO, gestão de redes sociais e o próprio lançamento. Na Hotmart, você pode cadastrar todas as pessoas envolvidas em um infoproduto e o percentual de suas partic

In [6]:
vectorstore.similarity_search("como vender com a hotmart?", k=2)

[Document(metadata={'source': 'hotmart'}, page_content='Como funciona a Hotmart?\n\nA Hotmart é uma plataforma de infoprodutos, que oferece diferentes recursos e\nsoluções para quem quer vender e comprar produtos digitais na internet.\n\nDe maneira resumida, a Hotmart funciona da seguinte maneira:\n\n  1. Uma pessoa com conhecimento em determinada área cria um infoproduto (como um curso online ou um ebook) para compartilhar sua habilidade com o mundo.\n  2. Pessoas com um bom relacionamento e facilidade em fazer vendas ajudam a divulgar e a vender esse produto para o consumidor em troca de uma comissão por venda realizada.\n  3. O consumidor que está em busca de um novo conhecimento pode comprar o produto em nossa plataforma, pesquisando na página de busca ou seguindo o Link de Divulgação (Hotlink) do Afiliado ou do Produtor.\n\nEm outras palavras, a Hotmart oferece uma plataforma EAD que hospeda cursos\nonline e processa todos os pagamentos online em um só lugar, além de oferecer\nsol

## Grid Search Chunking

In [ ]:
import os
import time
import nest_asyncio
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.evaluation import DatasetGenerator, FaithfulnessEvaluator, RelevancyEvaluator
from llama_index.llms.openai import OpenAI
from langchain_core.documents import Document
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from llama_index.readers.web import SimpleWebPageReader
import openai
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:1b", request_timeout=120.0)

nest_asyncio.apply()

faithfulness_gpt4 = FaithfulnessEvaluator(llm=llm)
relevancy_gpt4 = RelevancyEvaluator(llm=llm)

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://hotmart.com/pt-br/blog/como-funciona-hotmart"]
)
data_generator = DatasetGenerator.from_documents(documents)
eval_questions = data_generator.generate_questions_from_nodes()
eval_documents = documents[:20]

def evaluate_response_time_and_accuracy(chunk_size):
    total_response_time = 0
    total_faithfulness = 0
    total_relevancy = 0
    vector_index = VectorStoreIndex.from_documents(eval_documents,llm=llm)
    query_engine = vector_index.as_query_engine()
    num_questions = len(eval_questions)

    for question in eval_questions:
        start_time = time.time()
        response_vector = query_engine.query(question)
        elapsed_time = time.time() - start_time
        
        faithfulness_result = faithfulness_gpt4.evaluate_response(response=response_vector).passing
        relevancy_result = relevancy_gpt4.evaluate_response(query=question, response=response_vector).passing

        total_response_time += elapsed_time
        total_faithfulness += faithfulness_result
        total_relevancy += relevancy_result

    average_response_time = total_response_time / num_questions
    average_faithfulness = total_faithfulness / num_questions
    average_relevancy = total_relevancy / num_questions

    return average_response_time, average_faithfulness, average_relevancy

for chunk_size in [128, 256, 512, 1024, 2048]:
    avg_time, avg_faithfulness, avg_relevancy = evaluate_response_time_and_accuracy(chunk_size)
    print(f"Chunk size {chunk_size} - Average Response time: {avg_time:.2f}s, Average Faithfulness: {avg_faithfulness:.2f}, Average Relevancy: {avg_relevancy:.2f}")

